In [33]:
import transformers
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import string
import torch.nn as nn
from transformers import BertPreTrainedModel, BertConfig
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from time import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [8]:
# Load the dataset
df = pd.read_excel("C:\\Users\\suraj\\OneDrive\\Desktop\\DATASETS\\NewCleaned.xlsx")

In [9]:
# Check for null values
null_counts = df.isnull().any()
print (null_counts) 

label    False
texts    False
dtype: bool


In [10]:
# Replace non-alphabetic characters with space
df['cleaned_text'] = df['texts'].replace('[^a-zA-Z]', ' ', regex=True)

In [11]:
# Clean the text data
def clean_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stop words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Remove URLs
    import re
    text = re.sub(r"http\S+", "", text)
    # Remove extra whitespaces
    text = ' '.join(text.split())
    text = re.sub(r"â€™", "'", text)  # Replace â€™ with '
    return text

df["new_texts"] = df["cleaned_text"].apply(clean_text)

In [12]:
df.head()

,label,texts,cleaned_text,new_texts
0,0,A whirlwind day in D.C. showcases TrumpÃ¢Â€Â™s...,A whirlwind day in D C showcases Trump s...,whirlwind day c showcases trump unorthodox vie...
1,0,"In Baltimore's call for federal police probe, ...",In Baltimore s call for federal police probe ...,baltimore call federal police probe new search...
2,1,Trump Proudly Declares: Most Of The People IÃ¢...,Trump Proudly Declares Most Of The People I ...,trump proudly declares people insulted deserve...
3,0,Inside the Trump-Bush melodrama: Decades of te...,Inside the Trump Bush melodrama Decades of te...,inside trump bush melodrama decades tension di...
4,0,Shutdown clash to return in force by December ...,Shutdown clash to return in force by December ...,shutdown clash return force december notable n...


In [13]:
label_counts = df['label'].value_counts()
print("Count of 0s:", label_counts[0])
print("Count of 1s:", label_counts[1])

Count of 0s: 42760
Count of 1s: 42760


In [14]:
# Split the dataset into train, validation, and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [15]:
# Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the max length for the input texts
max_length = 512

# Tokenize and encode the texts
train_encodings = tokenizer(list(train_data['new_texts'].values), truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(list(val_data['new_texts'].values), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(list(test_data['new_texts'].values), truncation=True, padding=True, max_length=max_length)

In [19]:
# Create PyTorch tensors for the input data
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_data['label'].values))

val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            torch.tensor(val_data['label'].values))

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             torch.tensor(test_data['label'].values))



In [20]:
# Set the batch size
batch_size = 16

In [21]:
# Define the max length for the input texts
max_length = 512

# Tokenize and encode the texts
train_encodings = tokenizer(list(train_data['texts'].values), truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(list(val_data['texts'].values), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(list(test_data['texts'].values), truncation=True, padding=True, max_length=max_length)


In [24]:
# Create data loaders
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


In [25]:
# Load the custom BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [29]:
num_epochs = 3

# Set the optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * num_epochs # Number of training epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [30]:
# Initialize lists to store accuracy history
train_acc_history = []
val_acc_history = []

In [31]:
for epoch in range(num_epochs):
    model.train()
    train_pbar = tqdm(train_loader, position=0, leave=True)
    start_time = time()
    train_preds = []
    train_labels = []
    for batch in train_pbar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        model.zero_grad()

        preds = torch.argmax(outputs.logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
        train_pbar.set_postfix(loss=loss.item())

    train_accuracy = accuracy_score(train_labels, train_preds)
    train_acc_history.append(train_accuracy)

    # Evaluation on the validation set
    model.eval()
    val_pbar = tqdm(val_loader, position=0, leave=True)
    val_preds = []
    val_labels = []
    for batch in val_pbar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        preds = torch.argmax(logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    val_acc_history.append(val_accuracy)

    end_time = time()
    epoch_time = end_time - start_time
    remaining_time = epoch_time * (num_epochs - epoch - 1)
    print(f'Epoch {epoch + 1}/{num_epochs} - Time: {epoch_time:.2f}s - Remaining: {remaining_time:.2f}s')
    print(f'Training Accuracy: {train_accuracy:.4f} - Validation Accuracy: {val_accuracy:.4f}')

# Calculate average training and validation accuracy
avg_train_acc = sum(train_acc_history) / num_epochs
avg_val_acc = sum(val_acc_history) / num_epochs
print(f'Average Training Accuracy: {avg_train_acc:.4f}')
print(f'Average Validation Accuracy: {avg_val_acc:.4f}')   

100%|██████████| 856/856 [05:17<00:00,  2.69it/s]


Epoch 1/3 - Time: 3864.21s - Remaining: 7728.42s
Training Accuracy: 0.9635 - Validation Accuracy: 0.9800


100%|██████████| 856/856 [04:18<00:00,  3.31it/s]


Epoch 2/3 - Time: 3622.07s - Remaining: 3622.07s
Training Accuracy: 0.9886 - Validation Accuracy: 0.9863


100%|██████████| 856/856 [04:43<00:00,  3.02it/s]

Epoch 3/3 - Time: 3612.18s - Remaining: 0.00s
Training Accuracy: 0.9967 - Validation Accuracy: 0.9871
Average Training Accuracy: 0.9829
Average Validation Accuracy: 0.9845


In [34]:
# Evaluation on the test set
model.eval()
test_pbar = tqdm(test_loader, position=0, leave=True)
test_preds = []
test_labels = []
for batch in test_pbar:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    preds = torch.argmax(logits, dim=1)
    test_preds.extend(preds.cpu().numpy())
    test_labels.extend(labels.cpu().numpy())

# Compute overall accuracy
test_accuracy = accuracy_score(test_labels, test_preds)

# Compute per-class metrics
test_precision = precision_score(test_labels, test_preds, average=None)
test_recall = recall_score(test_labels, test_preds, average=None)
test_f1 = f1_score(test_labels, test_preds, average=None)

# Print per-class metrics
print('Test Results (Per Class):')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Test F1-Score: {test_f1}')

# Print overall metrics
print('\nOverall Test Results:')
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test F1-Score (Macro): {f1_score(test_labels, test_preds, average="macro"):.4f}')
print(f'Test Precision (Macro): {precision_score(test_labels, test_preds, average="macro"):.4f}')
print(f'Test Recall (Macro): {recall_score(test_labels, test_preds, average="macro"):.4f}')

100%|██████████| 1069/1069 [06:06<00:00,  2.92it/s]

Test Results (Per Class):
Test Precision: [0.98662913 0.98962462]
Test Recall: [0.98953064 0.98674881]
Test F1-Score: [0.98807776 0.98818462]

Overall Test Results:
Test Accuracy: 0.9881
Test F1-Score (Macro): 0.9881
Test Precision (Macro): 0.9881
Test Recall (Macro): 0.9881


In [35]:
import os
from transformers import BertForSequenceClassification

# Fixed path for saving the model
save_dir = "E:\\Major Project\\Models"

# Prompt the user for the model name
model_name = input("Enter the desired name for the model: ")

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_path = os.path.join(save_dir, model_name)
model_to_save.save_pretrained(model_path)

# Save the tokenizer
tokenizer.save_pretrained(model_path)

print(f'Model saved to {model_path}')

Enter the desired name for the model:  PT08v2


Model saved to E:\Major Project\Models\PT08v2


In [37]:
# Save results to a text file
import os

# Fixed path for saving the results file
results_file_path = "E:\\Major Project\\Models\\updates"

# Prompt the user for the file name
results_file_name = input("Name: ")

# Create the directory if it doesn't exist
os.makedirs(results_file_path, exist_ok=True)

# Create the full file path
results_file = os.path.join(results_file_path, results_file_name)

# Open the file in write mode
with open(results_file, 'w') as f:
    # Write the training accuracy history
    f.write('Training Accuracy History:\n')
    for epoch, acc in enumerate(train_acc_history):
        f.write(f'Epoch {epoch + 1}: {acc:.4f}\n')
    f.write('\n')

    # Write the validation accuracy history
    f.write('Validation Accuracy History:\n')
    for epoch, acc in enumerate(val_acc_history):
        f.write(f'Epoch {epoch + 1}: {acc:.4f}\n')
    f.write('\n')

    # Write the average training and validation accuracy
    f.write(f'Average Training Accuracy: {avg_train_acc:.4f}\n')
    f.write(f'Average Validation Accuracy: {avg_val_acc:.4f}\n\n')

    # Write the test results
    f.write('Test Results:\n')
    f.write(f'Test Accuracy: {test_accuracy:.4f}\n')
    f.write(f'Test F1-Score: {test_f1.mean():.4f}\n')  # Convert to scalar
    f.write(f'Test Precision: {test_precision.mean():.4f}\n')  # Convert to scalar
    f.write(f'Test Recall: {test_recall.mean():.4f}\n\n')  # Convert to scalar

    # Write the per-class test results
    f.write('Test Results (Per Class):\n')
    f.write('Test Precision: ')
    for value in test_precision:
        f.write(f'{value:.4f} ')
    f.write('\n')
    f.write('Test Recall: ')
    for value in test_recall:
        f.write(f'{value:.4f} ')
    f.write('\n')
    f.write('Test F1-Score: ')
    for value in test_f1:
        f.write(f'{value:.4f} ')
    f.write('\n')

print(f'Results saved to {results_file}')

Name:  Pt08v2 results april 23rd


Results saved to E:\Major Project\Models\updates\Pt08v2 results april 23rd
